In [2]:
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install accelerate>=0.20.1

Note: you may need to restart the kernel to use updated packages.


In [5]:
import datasets
dataset = datasets.load_dataset("csv", data_files="dataset_csv.csv", split='train')
print(dataset)

c:\Users\Aadil Sayad\Documents\Local Disk D (real)\Projects\V_Mail\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['prompt', 'label'],
    num_rows: 250
})


In [6]:
full_dataset = dataset.class_encode_column("label").train_test_split(test_size=0.2, stratify_by_column="label")

In [9]:
reloaded_dataset = full_dataset['train'].sort("label")

In [10]:
dummy_dataset = reloaded_dataset._select_contiguous(0, 8)
dummy_dataset = datasets.concatenate_datasets([dummy_dataset, reloaded_dataset._select_contiguous(40, 8)])
dummy_dataset = datasets.concatenate_datasets([dummy_dataset, reloaded_dataset._select_contiguous(80, 8)])
dummy_dataset = datasets.concatenate_datasets([dummy_dataset, reloaded_dataset._select_contiguous(120, 8)])
dummy_dataset = datasets.concatenate_datasets([dummy_dataset, reloaded_dataset._select_contiguous(160, 8)])

In [11]:
for i in range(8, 40, 8):
    dummy_dataset = datasets.concatenate_datasets([dummy_dataset, reloaded_dataset._select_contiguous(i, 8)])
    dummy_dataset = datasets.concatenate_datasets([dummy_dataset, reloaded_dataset._select_contiguous(40+i, 8)])
    dummy_dataset = datasets.concatenate_datasets([dummy_dataset, reloaded_dataset._select_contiguous(80+i, 8)])
    dummy_dataset = datasets.concatenate_datasets([dummy_dataset, reloaded_dataset._select_contiguous(120+i, 8)])
    dummy_dataset = datasets.concatenate_datasets([dummy_dataset, reloaded_dataset._select_contiguous(160+i, 8)])

In [12]:
validation_splits = [dummy_dataset._select_contiguous(k, 40) for k in range(0, 200, 40)]
train_splits = [datasets.concatenate_datasets([dummy_dataset._select_contiguous(0, k), dummy_dataset._select_contiguous(k+40 if k != 160 else 0, 200-k-40)]) for k in range(0, 200, 40)]

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding

In [14]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
training_args = TrainingArguments("test-trainer")

In [17]:
def tokenize_function(example):
    return tokenizer(example["prompt"], truncation=True)

In [18]:
tokenized_train_1 = train_splits[0].map(tokenize_function, batched=True)
tokenized_validation_1 = validation_splits[0].map(tokenize_function, batched=True)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_1,
    eval_dataset=tokenized_validation_1,
    data_collator=data_collator,
    tokenizer=tokenizer
)

Map: 100%|██████████| 40/40 [00:00<00:00, 881.20 examples/s]

In [19]:
trainer.train()

  0%|          | 0/60 [00:00<?, ?it/s]You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
